In [19]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors

In [20]:
# importar datos
# comments = pd.read_csv('Comments.csv')
courses = pd.read_csv('Course_info.csv')

In [21]:
# se eliminan las columnas que no interesan
# comments_filtered= comments.drop(labels=['date', 'display_name', 'comment'], axis=1)
column_keep = ['category', 'subcategory','instructor_name']
column_delete = []
for column in courses.columns:
    if column not in column_keep:
        column_delete.append(column)
courses_filtered = courses.drop(labels=column_delete, axis=1)

In [22]:
# comprueba la cantidad de valores nulos
print(courses_filtered.isnull().sum())
# crea una lista vacia para las columna con valores nulos
column_dropna=[]
# itera a traves de un dict nom_colum:cant_nulos y guarda el nombre de la columa si tiene nulos
for key, value in courses_filtered.isnull().sum().to_dict().items():
    if value !=0: 
        column_dropna.append(key)
# elina las filas que contienen nulos 
courses_filtered.dropna(axis= 0, how='any', subset=column_dropna, inplace= True)

category           0
subcategory        0
instructor_name    5
dtype: int64


In [23]:
# pipeline para variables numericas
# numeric_pipe = Pipeline([
#     ('scaler', StandardScaler())
#     ])

# pipeline para variables categoricas
categorical_pipe = Pipeline([
    ('encoder', OneHotEncoder(drop = 'first'))
    ])

# contiene las transformaciones
col_transf = ColumnTransformer(transformers=[
    #('numeric', numeric_pipe, courses_filtered._get_numeric_data().columns.tolist()),
    ('categoric', categorical_pipe, courses_filtered.select_dtypes('object').columns.tolist())
    ])

col_transf_fit = col_transf.fit(courses_filtered)
courses_filtered_transf = col_transf_fit.transform(courses_filtered)

nneighbors = NearestNeighbors(n_neighbors = 5, metric = 'cosine').fit(courses_filtered_transf)
# recomendaciones para las 10 primeras posiciones
ind = nneighbors.kneighbors(courses_filtered_transf[:10], return_distance=False)

In [24]:
#ejemplo de recomendacion
print("Curso visto")
print("-"*80)
print(courses_filtered.loc[ind[0][1], :])
print()
print("Cursos recomendados")
print("-"*80)
courses_filtered.loc[ind[0][1:], :]

Curso visto
--------------------------------------------------------------------------------
category                 Lifestyle
subcategory        Food & Beverage
instructor_name        Angela Poch
Name: 431, dtype: object

Cursos recomendados
--------------------------------------------------------------------------------


,category,subcategory,instructor_name
431,Lifestyle,Food & Beverage,Angela Poch
129102,Design,Architectural Design,D5 Render Team
131379,IT & Software,Other IT & Software,"Conecta, que eu te ensino!"
158163,IT & Software,IT Certifications,Rifat Academy


In [ ]:
#todo: unir courses id y title con los resultados del modelo

In [39]:
courses_filtered.loc[ind[0][1:], :]

,category,subcategory,instructor_name
431,Lifestyle,Food & Beverage,Angela Poch
129102,Design,Architectural Design,D5 Render Team
131379,IT & Software,Other IT & Software,"Conecta, que eu te ensino!"
158163,IT & Software,IT Certifications,Rifat Academy


In [26]:
courses[['id', 'title']].iloc[431]

id                                     26438.0
title    Happy Vegetarian Kids Cooking Healthy
Name: 431, dtype: object